In [7]:
import requests
import csv
import pandas as pd
from datetime import datetime, timedelta

# File path for the existing weather data
csv_filename = 'Year_weather.csv'

# Step 1: Load the CSV and find the latest date
def get_latest_date_from_csv(file_path):
    try:
        df = pd.read_csv(file_path)
        df['datetime'] = pd.to_datetime(df['datetime'])  # Ensure the datetime column is in datetime format
        latest_date = df['datetime'].max().date()  # Get the latest date in the file
        return latest_date
    except FileNotFoundError:
        # If the file doesn't exist, return None to indicate no previous data
        return None

# Step 2: Perform the API request from the latest date to today's date
def fetch_weather_data(start_date, end_date, output_file):
    # Format the dates for the API request (YYYY-MM-DD)
    start_date_str = start_date.strftime('%Y-%m-%d')
    end_date_str = end_date.strftime('%Y-%m-%d')

    # Weather API URL with dynamic dates
    url = f"https://weather.visualcrossing.com/VisualCrossingWebServices/rest/services/timeline/Boston%2CUnited%20States/{start_date_str}/{end_date_str}?unitGroup=us&elements=datetime%2Ctemp%2Cdew%2Chumidity%2Cprecip%2Cpreciptype%2Csnow%2Csnowdepth%2Cwindgust%2Cwindspeed%2Cwinddir%2Cpressure%2Ccloudcover%2Csolarradiation%2Csolarenergy%2Cuvindex%2Csevererisk&include=hours&key=SRGBWC4W94EFX6RMJXBC4L6EN&contentType=csv"
    
    response = requests.get(url)

    if response.status_code != 200:
        print('Unexpected Status code:', response.status_code)
        return

    # Append the new data to the existing CSV file
    with open(output_file, mode='a', newline='') as file:  # Open in append mode
        writer = csv.writer(file)
        csv_reader = csv.reader(response.text.splitlines(), delimiter=',', quotechar='"')

        # Skip the first row (header)
        next(csv_reader)

        # Write the remaining rows to the file
        for row in csv_reader:
            writer.writerow(row)

    print(f"Weather data from {start_date_str} to {end_date_str} has been appended to '{output_file}'")
# Step 3: Main function to coordinate the process
def main():
    # Get today's date
    today = datetime.today().date()

    # Get the latest date from the existing CSV file
    latest_date = get_latest_date_from_csv(csv_filename)

    if latest_date is None:
        # If no existing data, start from a default date (e.g., one year ago)
        start_date = today - timedelta(days=365)
    else:
        # Start fetching data from the day after the latest date in the CSV
        start_date = latest_date + timedelta(days=1)

    # Fetch and append weather data from the latest date to today
    if start_date <= today:
        fetch_weather_data(start_date, today, csv_filename)
    else:
        print("No new data to fetch, the CSV is already up to date.")

# Run the main function
if __name__ == "__main__":
    main()


Weather data from 2024-10-02 to 2024-10-25 has been appended to 'Year_weather.csv'
